In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv
/kaggle/input/cpc-codes/titles.csv
/kaggle/input/aigl-patentsberta-kaggle-v3/config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/training_args.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer.json
/kaggle/input/aigl-patentsberta-kaggle-v3/tokenizer_config.json
/kaggle/input/aigl-patentsberta-kaggle-v3/pytorch_model.bin
/kaggle/input/aigl-patentsberta-kaggle-v3/special_tokens_map.json
/kaggle/input/aigl-patentsberta-kaggle-v3/vocab.txt


# Credits:
this notebook is based on the notbook ["Getting started with NLP for absolute beginners"](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners) by @Jeremy Howard

## Imports

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns
import matplotlib.pyplot as plt

# EDA

In [3]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
print(train.shape)
train.head()

(36473, 5)


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [4]:
train.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
train.groupby('score').count()

,id,anchor,target,context
score,,,,
0.00,7471,7471,7471,7471
0.25,11519,11519,11519,11519
0.50,12300,12300,12300,12300
0.75,4029,4029,4029,4029
1.00,1154,1154,1154,1154


## Score meanings according to [Data Description](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data?select=train.csv)

The scores are in the 0-1 range with increments of 0.25 with the following meanings:

    1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
    0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
    0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
    0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
    0.0 - Unrelated.


# Test Corpus

In [6]:
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")

In [7]:
test.shape

(36, 4)

In [8]:
def cosine(u, v):
    """
    cosine similarity definition
    """
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# External Context

In [9]:
# ajouter le contexte https://www.cooperativepatentclassification.org/Archive
# extraire le contexte du fichier xml

In [10]:
titles = pd.read_csv("../input/cpc-codes/titles.csv")
print(titles.shape)
titles.head()

(260476, 7)


,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


# HuggingFace AutoModelForSequenceClassification

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr # pour trouver les correspondances entre les résultats sur des échelles variées
from sklearn.model_selection import StratifiedGroupKFold

In [12]:
import transformers
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

import torch
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [13]:
import warnings
import logging
from IPython.display import display, HTML

# Corpora merge

In [14]:
train['section'] = train["context"].str[0]
print(train.shape)
display(train.head())
titles=titles.rename(columns={"code": "context"})
display(titles.head())
train = train.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
train = train.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(train.head())

(36473, 6)


,id,anchor,target,context,score,section
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,score,section,context_title,section_title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,HUMAN NECESSITIES


In [15]:
logging.disable(logging.WARNING)
warnings.simplefilter('ignore')

# Deberta-v3-small

In [16]:
#model_name = 'AI-Growth-Lab/PatentSBERTa_Kaggle_V3' #(online version)
model_name = '../input/aigl-patentsberta-kaggle-v3'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
tokenizer.all_special_tokens

['<s>', '</s>', '[UNK]', '<pad>', '<mask>']

In [19]:
def clean(x):
    t = x.lower()
    t = t.replace("[",'')
    t = t.replace(";",'')
    t = t.replace(",",'')
    t = t.replace("]",'')
    t = t.replace(":",'')
    t = t.replace("(",'')
    t = t.replace(")",'')
    t = t.replace("{",'')
    t = t.replace("}",'')
    t = t.replace("/",' ')
    t = t.replace("-",' ')
    return t

In [20]:
train["inputs"] = train["section_title"].apply(clean) + " [SEP] " + train["anchor"] + " [SEP] " + train["target"]

In [21]:
def tok_func(x):
    return tokenizer(x["inputs"])

In [22]:
train_ds = Dataset.from_pandas(train).rename_column('score', 'label')

In [23]:
inps = "anchor","target","context","context_title","section_title"
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/37 [00:00<?, ?ba/s]

In [24]:
tok_ds

Dataset({
    features: ['label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [25]:
# bs - batch size
lr,bs = 8e-5,64
# wd: weight decay
wd,epochs = 0.01,4

In [26]:
anchors = train["anchor"].unique()
np.random.shuffle(anchors)

In [27]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [28]:
is_val = np.isin(train.anchor, val_anchors)
idxs = np.arange(len(train))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

(9358, 27115)

In [29]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [30]:
def corr(eval_pred): 
    logits, labels = eval_pred
    logits = logits.reshape(-1)
    return {'pearson': np.corrcoef(logits, labels)[0][1]}

In [31]:
#args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    #evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
     #num_train_epochs=epochs, weight_decay=wd, report_to='none')
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, ignore_mismatched_sizes=True)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr)

In [33]:
dds['train'][0]

{'label': 0.5,
 '__index_level_0__': 0,
 'input_ids': [0,
  2533,
  26789,
  7975,
  6451,
  1035,
  19806,
  1037,
  19561,
  18536,
  4769,
  1035,
  19806,
  1037,
  19561,
  18536,
  4769,
  2001,
  10800,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.021913,0.878728
2,0.015400,0.014768,0.883138
3,0.011200,0.015245,0.891145
4,0.007100,0.014845,0.893373


TrainOutput(global_step=1696, training_loss=0.010615652883952519, metrics={'train_runtime': 378.2473, 'train_samples_per_second': 286.744, 'train_steps_per_second': 4.484, 'total_flos': 1367150796214716.0, 'train_loss': 0.010615652883952519, 'epoch': 4.0})

In [35]:
trainer.save_model()

# Test corpus preprocessing

In [36]:
test.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [37]:
test['section'] = test["context"].str[0]
print(test.shape)
display(titles.head())
test = test.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
test = test.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(test.head())

(36, 5)


,context,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


,id,anchor,target,context,section,context_title,section_title
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G,OPTICS,PHYSICS
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G,OPTICS,PHYSICS
2,7aa5908a77a7ec24,el display,illumination,G02,G,OPTICS,PHYSICS
3,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11,G,INFORMATION STORAGE,PHYSICS
4,12ca31f018a2e2b9,speed control means,control loop,G05,G,CONTROLLING; REGULATING,PHYSICS


In [38]:
test["inputs"] = test["section_title"].apply(clean) + " [SEP] " + test["anchor"] + " [SEP] " + test["target"]

In [39]:
test_ds = Dataset.from_pandas(test)

In [40]:
test_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'section', 'context_title', 'section_title', 'inputs', '__index_level_0__'],
    num_rows: 36
})

In [41]:
inps = "anchor","target","context","context_title","section_title"
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
preds = trainer.predict(test_tok_ds).predictions.astype(float)
preds

array([[ 0.49192902],
       [ 1.01180053],
       [ 0.27321777],
       [ 0.50883985],
       [ 0.15532695],
       [ 0.79315227],
       [ 0.52949548],
       [ 0.75912106],
       [ 1.05208457],
       [ 0.26867014],
       [ 0.268154  ],
       [ 0.53384495],
       [-0.0079232 ],
       [ 0.56982422],
       [ 0.77778244],
       [-0.00385815],
       [ 0.19702075],
       [ 0.78184533],
       [ 0.7969408 ],
       [-0.0103711 ],
       [ 0.2803936 ],
       [ 0.23474042],
       [-0.00709068],
       [ 0.28545764],
       [-0.00724544],
       [ 0.48683783],
       [ 0.31521079],
       [ 0.53596586],
       [ 0.05472084],
       [ 0.48354414],
       [ 0.47155106],
       [ 0.20992929],
       [-0.00682015],
       [ 0.22267312],
       [-0.00698859],
       [-0.00766702]])

In [43]:
type(preds)

numpy.ndarray

In [44]:
df_submission = pd.DataFrame()

In [45]:
df_submission['id'] = test['id'].copy()

In [46]:
df_submission

,id
0,4112d61851461f60
1,5203a36c501f1b7c
2,7aa5908a77a7ec24
3,442c114ed5c4e3c9
4,12ca31f018a2e2b9
5,9f2279ce667b21dc
6,b9ea2b06a878df6f
7,09e418c93a776564
8,ae0262c02566d2ce
9,03ba802ed4029e4d


In [47]:
preds_list = preds.tolist()

In [48]:
preds_list

[[0.4919290244579315],
 [1.0118005275726318],
 [0.2732177674770355],
 [0.5088398456573486],
 [0.1553269475698471],
 [0.7931522727012634],
 [0.5294954776763916],
 [0.7591210603713989],
 [1.0520845651626587],
 [0.26867014169692993],
 [0.26815399527549744],
 [0.5338449478149414],
 [-0.00792319979518652],
 [0.56982421875],
 [0.7777824401855469],
 [-0.0038581511471420527],
 [0.1970207542181015],
 [0.7818453311920166],
 [0.796940803527832],
 [-0.010371102951467037],
 [0.2803936004638672],
 [0.23474042117595673],
 [-0.007090682163834572],
 [0.28545764088630676],
 [-0.007245441898703575],
 [0.48683783411979675],
 [0.3152107894420624],
 [0.5359658598899841],
 [0.054720837622880936],
 [0.48354414105415344],
 [0.4715510606765747],
 [0.20992928743362427],
 [-0.006820147857069969],
 [0.22267311811447144],
 [-0.0069885943084955215],
 [-0.007667019963264465]]

In [49]:
flat_list = [x for xs in preds_list for x in xs]
flat_list

[0.4919290244579315,
 1.0118005275726318,
 0.2732177674770355,
 0.5088398456573486,
 0.1553269475698471,
 0.7931522727012634,
 0.5294954776763916,
 0.7591210603713989,
 1.0520845651626587,
 0.26867014169692993,
 0.26815399527549744,
 0.5338449478149414,
 -0.00792319979518652,
 0.56982421875,
 0.7777824401855469,
 -0.0038581511471420527,
 0.1970207542181015,
 0.7818453311920166,
 0.796940803527832,
 -0.010371102951467037,
 0.2803936004638672,
 0.23474042117595673,
 -0.007090682163834572,
 0.28545764088630676,
 -0.007245441898703575,
 0.48683783411979675,
 0.3152107894420624,
 0.5359658598899841,
 0.054720837622880936,
 0.48354414105415344,
 0.4715510606765747,
 0.20992928743362427,
 -0.006820147857069969,
 0.22267311811447144,
 -0.0069885943084955215,
 -0.007667019963264465]

In [50]:
df_submission['score' ] = flat_list

In [51]:
df_submission

,id,score
0,4112d61851461f60,0.491929
1,5203a36c501f1b7c,1.011801
2,7aa5908a77a7ec24,0.273218
3,442c114ed5c4e3c9,0.508840
4,12ca31f018a2e2b9,0.155327
5,9f2279ce667b21dc,0.793152
6,b9ea2b06a878df6f,0.529495
7,09e418c93a776564,0.759121
8,ae0262c02566d2ce,1.052085
9,03ba802ed4029e4d,0.268670


In [52]:
#submission = datasets.Dataset.from_dict({
  #  'id': test_ds['id'],
 #   'score': preds
#})

df_submission.to_csv('submission.csv', index=False)